In [2]:
import sys
import tkinter
import tkinter.messagebox
from tkintermapview import TkinterMapView
from pyswip import Prolog
import pandas as pd

In [3]:
class App(tkinter.Tk):

    APP_NAME = "map_view_demo.py"
    WIDTH = 800
    HEIGHT = 750  # This is now the initial size, not fixed.

    def __init__(self, *args, **kwargs):
        tkinter.Tk.__init__(self, *args, **kwargs)

        self.title(self.APP_NAME)
        self.geometry(f"{self.WIDTH}x{self.HEIGHT}")

        # Configure the grid
        self.grid_columnconfigure(0, weight=1)
        self.grid_rowconfigure(0, weight=1)  # Text area and submit button combined row
        self.grid_rowconfigure(1, weight=4)  # Map row

        # Upper part: Text Area and Submit Button
        self.text_area = tkinter.Text(self, height=5)  # Reduced height for text area
        self.text_area.grid(row=0, column=0, pady=(10, 0), padx=10, sticky="nsew")

        self.submit_button = tkinter.Button(self, text="Submit", command=self.process_text)
        self.submit_button.grid(row=0, column=0, pady=(0, 10), padx=10, sticky="se")  # Placed within the same cell as text area

        # Lower part: Map Widget
        self.map_widget = TkinterMapView(self)
        self.map_widget.grid(row=1, column=0, sticky="nsew")

        self.marker_list = []  # Keeping track of markers
        self.marker_path = None


    def __init__(self, *args, **kwargs):
        tkinter.Tk.__init__(self, *args, **kwargs)

        self.title(self.APP_NAME)
        self.geometry(f"{self.WIDTH}x{self.HEIGHT}")

        # Configure the grid
        self.grid_columnconfigure(0, weight=1)
        self.grid_rowconfigure(0, weight=1)  # Text area can expand/contract.
        self.grid_rowconfigure(1, weight=0)  # Submit button row; doesn't need to expand.
        self.grid_rowconfigure(2, weight=3)  # Map gets the most space.

        # Upper part: Text Area and Submit Button
        self.text_area = tkinter.Text(self)
        self.text_area.grid(row=0, column=0, pady=10, padx=10, sticky="nsew")
        
        self.submit_button = tkinter.Button(self, text="Submit", command=self.process_text)
        self.submit_button.grid(row=1, column=0, pady=10, sticky="ew")

        # Lower part: Map Widget
        self.map_widget = TkinterMapView(self)
        self.map_widget.grid(row=2, column=0, sticky="nsew")

        self.marker_list = []  # Keeping track of markers

    def check_connections(self, results):
        print('result2 ', results)
        locations = []
        for result in results:
            city  = result["City"]
            locations.append(city)
            # TODO 5: create the knowledgebase of the city and its connected destinations using Adjacency_matrix.csv


        return locations

    def process_text(self):
        """Extract locations from the text area and mark them on the map."""
        text = self.text_area.get("1.0", "end-1c")  # Get text from text area
        locations = self.extract_locations(text)  # Extract locations (you may use a more complex method here)


        # TODO 4: create the query based on the extracted features of user desciption 
        ################################################################################################
        query = "destination(City,_, _, _, low, _, _, _, _, _, _, _, _)"
        results = list(prolog.query(query))
        print(results)
        locations = self.check_connections(results)
        # TODO 6: if the number of destinations is less than 6 mark and connect them 
        ################################################################################################
        print(locations)
        locations = ['mexico_city','rome' ,'brasilia']
        self.mark_locations(locations)

    def mark_locations(self, locations):
        """Mark extracted locations on the map."""
        for address in locations:
            marker = self.map_widget.set_address(address, marker=True)
            if marker:
                self.marker_list.append(marker)
        self.connect_marker()
        self.map_widget.set_zoom(1)  # Adjust as necessary, 1 is usually the most zoomed out


    def connect_marker(self):
        print(self.marker_list)
        position_list = []

        for marker in self.marker_list:
            position_list.append(marker.position)

        if hasattr(self, 'marker_path') and self.marker_path is not None:
            self.map_widget.delete(self.marker_path)

        if len(position_list) > 0:
            self.marker_path = self.map_widget.set_path(position_list)

    def extract_locations(self, text):
        """Extract locations from text. A placeholder for more complex logic."""
        # Placeholder: Assuming each line in the text contains a single location name
        # TODO 3: extract key features from user's description of destinations
        ################################################################################################

        return [line.strip() for line in text.split('\n') if line.strip()]

    def start(self):
        self.mainloop()


In [6]:
def sanatize(string):
    return string.lower().replace('\'', '').replace('.', '').split(',')[0]

In [13]:
# TODO 1: read destinations' descriptions from Destinations.csv and add them to the prolog knowledge base
################################################################################################
df = pd.read_csv('Destinations.csv')

df.head()

,Destinations,country,region,Climate,Budget,Activity,Demographics,Duration,Cuisine,History,Natural Wonder,Accommodation,Language
0,Tokyo,Japan,East Asia,Temperate,High,Cultural,Solo,Long,Asian,Modern,Mountains,Luxury,Japanese
1,Ottawa,Canada,North America,Cold,Medium,Adventure,Family-friendly,Medium,European,Modern,Forests,Mid-range,English
2,Mexico City,Mexico,North America,Temperate,Low,Cultural,Senior,Short,Latin American,Ancient,Mountains,Budget,Spanish
3,Rome,Italy,Southern Europe,Temperate,High,Cultural,Solo,Medium,European,Ancient,Beaches,Luxury,Italian
4,Brasilia,Brazil,South America,Tropical,Low,Adventure,Family-friendly,Long,Latin American,Modern,Beaches,Budget,Portuguese


In [14]:
prolog = Prolog()
prolog.retractall("destination(_, _, _, _, _, _, _, _, _, _, _, _, _)")

for data in df.values:
    assert_str = ''
    for i in range(len(data)):
        
        assert_str += '\'' + sanatize(data[i]) + '\''
        
        if i != len(data)-1:
            assert_str += ', '    
            
    prolog.assertz(f"destination({assert_str})")

In [8]:
# TODO 2: extract unique features from the Destinations.csv and save them in a dictionary
################################################################################################
city_info = {}

query = 'destination(City, _, _, _, low, _, _, _, _, _, _, _, _)'
results = list(prolog.query(query))

for result in results:
    print(result["City"])
    
print(len(results))

mexico city
brasilia
new delhi
moscow
bangkok
cairo
buenos aires
kathmandu
nairobi
jakarta
hanoi
rabat
lima
manila
kuala lumpur
bogota
warsaw
kyiv
dhaka
islamabad
colombo
tehran
shiraz
mashhad
yazd
chengdu
cusco
arequipa
medellin
cali
lahore
karachi
izmir
bursa
alexandria
luxor
aswan
37


In [9]:
if __name__ == "__main__":
    app = App()
    app.start()

[{'City': 'mexico city'}, {'City': 'brasilia'}, {'City': 'new delhi'}, {'City': 'moscow'}, {'City': 'bangkok'}, {'City': 'cairo'}, {'City': 'buenos aires'}, {'City': 'kathmandu'}, {'City': 'nairobi'}, {'City': 'jakarta'}, {'City': 'hanoi'}, {'City': 'rabat'}, {'City': 'lima'}, {'City': 'manila'}, {'City': 'kuala lumpur'}, {'City': 'bogota'}, {'City': 'warsaw'}, {'City': 'kyiv'}, {'City': 'dhaka'}, {'City': 'islamabad'}, {'City': 'colombo'}, {'City': 'tehran'}, {'City': 'shiraz'}, {'City': 'mashhad'}, {'City': 'yazd'}, {'City': 'chengdu'}, {'City': 'cusco'}, {'City': 'arequipa'}, {'City': 'medellin'}, {'City': 'cali'}, {'City': 'lahore'}, {'City': 'karachi'}, {'City': 'izmir'}, {'City': 'bursa'}, {'City': 'alexandria'}, {'City': 'luxor'}, {'City': 'aswan'}]
result2  [{'City': 'mexico city'}, {'City': 'brasilia'}, {'City': 'new delhi'}, {'City': 'moscow'}, {'City': 'bangkok'}, {'City': 'cairo'}, {'City': 'buenos aires'}, {'City': 'kathmandu'}, {'City': 'nairobi'}, {'City': 'jakarta'}, {'